# Retrieving drug-drug interactions from rxnav-in-the-box

In [2]:
import requests
import itertools
from collections import Counter
import json
from pathlib import Path
import pandas as pd

from tqdm import tqdm

In [2]:
rxnav_addr = 'http://192.168.68.117:4000'

## Getting all `rxterms` concepts

### Work with rxcui from `rxnorm_full`

In [17]:
rxColnames = [_.split('\t')[1] for _ in '''#	Field Name	Description	Representation
1	RXCUI	RxNorm unique identifier for concept (concept ID)	MRCONSO.SCUI if SAB = "RXNORM"; MRSAT.ATN= "RXCUI"
2	LAT	Language of term	Not processed
3	TS	Term status	Not processed
4	LUI	Unique identifier for term (no value provided)	Not applicable
5	STT	String type	Not processed
6	SUI	Unique identifier for string (no value provided)	Not applicable
7	ISPREF	Atom status - preferred (Y) or not (N) for this string within the concept	Not processed
8	RXAUI	Unique identifier for atom (RxNorm atom ID)	MRCONSO.SAUI if SAB = "RXNORM"; MRSAT.ATN = "RXAUI"
9	SAUI	Source asserted atom identifier	MRCONSO.SAUI
10	SCUI	Source asserted concept identifier	MRCONSO.SCUI
11	SDUI	Source asserted descriptor identifier	MRCONSO.SDUI
12	SAB	Abbreviated source name	MRCONSO.SAB
13	TTY	Abbreviation for term type	MRCONSO.TTY
14	CODE	Source asserted identifier	MRCONSO.CODE
15	STR	String	MRCONSO.STR
16	SRL	Source restriction level (no value provided)	Not applicable
17	SUPPRESS	Suppressible flag	MRCONSO.SUPPRESS
18	CVF	Content view flag	Not processed'''.split('\n')[1:]]

In [18]:
rxNorm_df = pd.read_table('RxNorm_full_09052023/rrf/RXNCONSO.RRF', 
                          delimiter='|', 
                          names = rxColnames,
                          index_col=False
                         )

/tmp/ipykernel_26200/2992394409.py:1: DtypeWarning: Columns (9,10,13) have mixed types. Specify dtype option on import or set low_memory=False.
  rxNorm_df = pd.read_table('RxNorm_full_09052023/rrf/RXNCONSO.RRF',


In [19]:
rxNorm_df

,RXCUI,LAT,TS,LUI,STT,SUI,ISPREF,RXAUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF
0,3,ENG,NaN,NaN,NaN,NaN,NaN,8717795,NaN,58488005,NaN,SNOMEDCT_US,PT,58488005,"1,4-alpha-Glucan branching enzyme",NaN,N,NaN
1,3,ENG,NaN,NaN,NaN,NaN,NaN,8717796,NaN,58488005,NaN,SNOMEDCT_US,FN,58488005,"1,4-alpha-Glucan branching enzyme (substance)",NaN,N,NaN
2,3,ENG,NaN,NaN,NaN,NaN,NaN,8717808,NaN,58488005,NaN,SNOMEDCT_US,SY,58488005,"Amylo-(1,4,6)-transglycosylase",NaN,N,NaN
3,3,ENG,NaN,NaN,NaN,NaN,NaN,8718164,NaN,58488005,NaN,SNOMEDCT_US,SY,58488005,Branching enzyme,NaN,N,NaN
4,19,ENG,NaN,NaN,NaN,NaN,NaN,10794494,NaN,112116001,NaN,SNOMEDCT_US,SY,112116001,17-hydrocorticosteroid,NaN,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1149033,2663690,ENG,NaN,NaN,NaN,NaN,NaN,12842539,12842539.0,2663690,NaN,RXNORM,SCDGP,2663690,chlorhexidine gluconate Oral Liquid Product,NaN,N,4096.0
1149034,2663691,ENG,NaN,NaN,NaN,NaN,NaN,12842540,12842540.0,2663691,NaN,RXNORM,SCDGP,2663691,diphenhydramine hydrochloride Oral Liquid Product,NaN,N,4096.0
1149035,2663691,ENG,NaN,NaN,NaN,NaN,NaN,12847789,12847789.0,2663691,NaN,RXNORM,TMSY,2663691,diphenhydrAMINE hydrochloride Oral Liquid Product,NaN,N,4096.0
1149036,2663692,ENG,NaN,NaN,NaN,NaN,NaN,12842541,12842541.0,2663692,NaN,RXNORM,SCDGP,2663692,chlophedianol hydrochloride / dexbromphenirami...,NaN,N,4096.0


In [21]:
rxNorm_df['RXCUI'].drop_duplicates()

0                3
4               19
8               21
11              22
14              33
            ...   
1149032    2663689
1149033    2663690
1149034    2663691
1149036    2663692
1149037    2663693
Name: RXCUI, Length: 386266, dtype: int64

In [26]:
rxcui_drugbank_code_df = rxNorm_df[
    rxNorm_df['SAB'] == 'DRUGBANK'
][['RXCUI', 'CODE']].drop_duplicates()

In [29]:
rxcui_codes_set = set(rxcui_drugbank_code_df['RXCUI'])
len(rxcui_codes_set)

11957

In [54]:
rxcui_codes_list = list([str(s) for s in rxcui_codes_set])
len(rxcui_codes_list)

11957

## Querying rxnav

In [55]:
responses = []
pattern = f'{rxnav_addr}/REST/interaction/interaction.json?rxcui='
save_every = 1000

for idx, k in enumerate(tqdm(rxcui_codes_list[:])):
    try:
        response = requests.get(f'{pattern}{k}')
        responses.append(json.loads(response.text))
    except Exception as e:
        print(e)
    if len(responses) == save_every:
        pd.to_pickle(
            responses,
            Path('query_results').joinpath(f'raw_rxnav_interact_resp_{idx+1}.pkl')
        )
        responses = []

pd.to_pickle(
    responses,
    Path('query_results').joinpath(f'raw_rxnav_interact_resp_{idx+1}.pkl')
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11957/11957 [19:27<00:00, 10.24it/s]


In [3]:
all_saved_responses_flist = list(
    Path('query_results/').glob('*.pkl')
)
len(all_saved_responses_flist)

12

In [4]:
all_saved_responses_list = []

In [5]:
interactions = []
for chunk_fname in tqdm(all_saved_responses_flist):
    responses = pd.read_pickle(chunk_fname)
    for resp in responses:
        if 'interactionTypeGroup' in resp:
            for resp_1 in resp['interactionTypeGroup']:
                for intType in resp_1['interactionType']:
                    for pair in intType['interactionPair']:
                        interactions.append(
                            {
                                'source': {
                                    'id': pair['interactionConcept'][0]['sourceConceptItem']['id'],
                                    'string': pair['interactionConcept'][0]['sourceConceptItem']['name'],
                                },
                                'target': {
                                    'id': pair['interactionConcept'][1]['sourceConceptItem']['id'],
                                    'string': pair['interactionConcept'][1]['sourceConceptItem']['name'],
                                },
                                'description': pair['description'],
                            }
                        )

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:39<00:00,  3.29s/it]


In [6]:
len(interactions)

2563520

In [7]:
interactions[:3]

[{'source': {'id': 'DB01185', 'string': 'Fluoxymesterone'},
  'target': {'id': 'DB00359', 'string': 'Sulfadiazine'},
  'description': 'Fluoxymesterone may increase the hypoglycemic activities of Sulfadiazine.'},
 {'source': {'id': 'DB01185', 'string': 'Fluoxymesterone'},
  'target': {'id': 'DB01015', 'string': 'Sulfamethoxazole'},
  'description': 'Fluoxymesterone may increase the hypoglycemic activities of Sulfamethoxazole.'},
 {'source': {'id': 'DB01185', 'string': 'Fluoxymesterone'},
  'target': {'id': 'DB00263', 'string': 'Sulfisoxazole'},
  'description': 'Fluoxymesterone may increase the hypoglycemic activities of Sulfisoxazole.'}]

In [8]:
interactions_list = []

for pair in interactions:
    interactions_list.append(
        {
            'source_drugbank_id': pair['source']['id'],
            'source_name': pair['source']['string'],
            'target_drugbank_id': pair['target']['id'],
            'target_name': pair['target']['string'],
            'description': pair['description']
        }
    )

In [9]:
interactions_df = pd.DataFrame(interactions_list).drop_duplicates()

In [10]:
interactions_df

,source_drugbank_id,source_name,target_drugbank_id,target_name,description
0,DB01185,Fluoxymesterone,DB00359,Sulfadiazine,Fluoxymesterone may increase the hypoglycemic ...
1,DB01185,Fluoxymesterone,DB01015,Sulfamethoxazole,Fluoxymesterone may increase the hypoglycemic ...
2,DB01185,Fluoxymesterone,DB00263,Sulfisoxazole,Fluoxymesterone may increase the hypoglycemic ...
3,DB01185,Fluoxymesterone,DB08962,Glibornuride,Fluoxymesterone may increase the hypoglycemic ...
4,DB01185,Fluoxymesterone,DB01382,Glymidine,Fluoxymesterone may increase the hypoglycemic ...
...,...,...,...,...,...
2563511,DB01764,Dalfopristin,DB01039,Fenofibrate,The metabolism of Fenofibrate can be decreased...
2563512,DB01764,Dalfopristin,DB06176,Romidepsin,The metabolism of Romidepsin can be decreased ...
2563513,DB01764,Dalfopristin,DB01238,Aripiprazole,The metabolism of Aripiprazole can be decrease...
2563514,DB01764,Dalfopristin,DB00908,Quinidine,The metabolism of Quinidine can be decreased w...


In [73]:
interactions_df.to_csv(
    'rxnav_in_the_box_05_Sep_2023_drug_interact.csv'
)